In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [23]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long, device='cuda:0')


In [24]:
training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 20
HIDDEN_DIM = 20

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [25]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [26]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
if torch.cuda.is_available():
   model.cuda()
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.1796, -1.2288, -0.9164],
        [-1.1790, -1.2737, -0.8852],
        [-1.1527, -1.2791, -0.9015],
        [-1.1316, -1.3766, -0.8556],
        [-1.1461, -1.4362, -0.8113]], device='cuda:0')


In [27]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [28]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.0179, -4.6662, -4.7910],
        [-5.1142, -0.0110, -5.3096],
        [-4.5490, -4.2161, -0.0257],
        [-0.0135, -5.1114, -4.9066],
        [-5.1400, -0.0090, -5.7637]], device='cuda:0')


In [29]:
sentence_in_ = prepare_sequence(training_data[0][0],  word_to_ix)
embeds_ = model.word_embeddings(sentence_in_)
embeds_.view(len(sentence_in_), 1, -1)

tensor([[[-1.4856e+00, -7.7408e-01, -7.0244e-01, -1.7687e+00, -1.7021e-01,
          -6.8308e-01, -9.6502e-01, -1.6634e+00, -6.9339e-01,  3.7438e-01,
          -7.4444e-01, -3.1240e-01, -2.0804e-01,  1.7792e+00,  2.1497e-01,
           4.7917e-01, -7.3632e-01, -1.1828e+00,  7.7857e-01,  2.9061e-01]],

        [[ 9.2322e-01,  2.1159e-01, -6.9439e-01,  8.9629e-01,  1.1434e+00,
          -2.0545e-01, -2.2900e+00, -1.4923e+00,  2.0564e-03, -7.7693e-01,
          -9.1681e-01, -7.3977e-02,  1.9754e+00, -6.4838e-02,  1.8536e-01,
          -6.6666e-01,  2.7596e-01,  1.1958e-01,  1.9108e-01, -5.1712e-01]],

        [[-1.1702e-01,  3.0066e-01, -1.3696e-01, -6.0383e-02,  2.5389e+00,
          -1.1621e+00,  1.6813e+00, -6.3682e-01, -9.3951e-01,  6.0543e-01,
           1.4675e-02, -5.0056e-01,  7.7515e-01,  4.4904e-01,  1.2905e+00,
           2.0718e+00,  1.8710e-02,  9.7104e-01, -1.4751e-01, -7.0730e-01]],

        [[ 1.5534e+00, -8.7924e-01, -3.4259e+00, -9.1230e-01, -7.4832e-02,
          -1.084

In [30]:
embeds_.shape

torch.Size([5, 20])

In [31]:
sentence_in

tensor([5, 6, 7, 8], device='cuda:0')

In [32]:
targets 

tensor([1, 2, 0, 1], device='cuda:0')

In [37]:
for sentence, tags in training_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = prepare_sequence(tags, tag_to_ix)
    print(model.word_embeddings(sentence_in).shape)
    print(targets.shape)

torch.Size([5, 20])
torch.Size([5])
torch.Size([4, 20])
torch.Size([4])
